In [ ]:
import os
from retinaface import RetinaFace
from deepface import DeepFace
from PIL import Image
import numpy as np
import shutil
class FaceSearchService:
    def __init__(self):
        self.image_cache = {}

    def load_images(self, images_folder):
        if images_folder not in self.image_cache:
            image_files = [os.path.join(images_folder, f) for f in os.listdir(images_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
            images = {}
            for image_path in image_files:
                try:
                    with Image.open(image_path) as img:
                        images[image_path] = img.copy()  # Hafızada bir kopyasını sakla
                except Exception as e:
                    print(f"Error loading image {image_path}: {e}")
            self.image_cache[images_folder] = images

    def face_recognition_on_image(self, image, db_path= "demoData/"):
        results = []
        try:
            image_np = np.array(image)
            faces = RetinaFace.detect_faces(image_np)
        except Exception as e:
            print("Face detection error:", e)
            return []

        if faces is None or not faces:
            print("No faces detected.")
            return []

        for face_id, face_info in faces.items():
            facial_area = face_info['facial_area']
            x, y, w, h = facial_area
            face_image = image.crop((x, y, w, h))
            if face_image.mode != 'RGB':
                face_image = face_image.convert('RGB')

            face_image.save("temp_face.jpg")
            try:
                result = DeepFace.find(img_path="temp_face.jpg", db_path=db_path, model_name='ArcFace', detector_backend='retinaface', enforce_detection=False, threshold=0.2)
                identity = result[0].iloc[0]['identity']
                person_name = os.path.basename(os.path.dirname(identity))
                person_distance = 1 - result[0].iloc[0]['distance']
                results.append({'name': person_name, 'distance': person_distance})
            except Exception as e:
                print("Recognition error:", e)
                results.append({'name': "No match found", 'distance': "N/A"})
        return results

    def face_search_retina(self, person_name, images_folder="content/uniq2"):
        self.load_images(images_folder)
        found_images = []
        for image_path, image in self.image_cache[images_folder].items():
            result = self.face_recognition_on_image(image)
            if result and any(person_name.lower() in res['name'].lower() for res in result):
                found_images.append(image_path)  # Resim yerine resim yolu ekleyin
        return found_images

if __name__ == '__main__':
    service = FaceSearchService()
    person_name = 'receptayyiperdogan'
    found_images = service.face_search_retina(person_name)
    print(f"Found {len(found_images)} images containing '{person_name}':")
    for i, image_path in enumerate(found_images, 1):

        print(f"Image {i}: {image_path}")  # Resim yolu yazdırılıyor